# usual imports

In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import re

In [9]:
def parse_amount(amount) :
    
    
    if 'g' not in amount :
        raise ValueError('The amount string should contain mg.')
       
    elif '%' not in amount :
        
        if len(amount.replace("µ", '')) != len(amount) or  len(re.sub('[μ]','',amount)) != len(amount):
            
            amount = re.sub('[μ]','',amount)
            amount = amount.replace("µ", '')
            return str(float(re.sub('[mgμ]','',amount).strip())/1000.0)
        
        elif 'm' not in amount :
            return str(float(re.sub('[mgμ]','',amount).strip())*1000.0)
        
        else :
            return re.sub('[mμg]','',amount).strip()
        
    
    else :
        
        to_parse = ""
        
        #determine if % is before or after g
        if amount.index('%') < amount.index('g') :
            to_parse = amount[amount.index('%')+1:]
        
        else :
            to_parse = amount[:amount.index('g')]
            
        if len(to_parse.replace("µ", '')) != len(to_parse) or len(re.sub('[μ]','',to_parse)) != len(to_parse):
            
            to_parse = to_parse.replace("µ", '')
            to_parse = re.sub('[μ]','',to_parse)
            return str(float(re.sub('[mgμ]','',to_parse).strip())/1000.0)
        
        elif 'm' not in to_parse :
            return str(float(re.sub('[mgμ]','',to_parse).strip())*1000.0)
        
        else :
            return re.sub('[mgμ]','',to_parse).strip()
        

# load wanted elements names

In [3]:
elements_names = pd.read_excel('data/RDI.xlsx')['element'].values
elements_names

array(['alpha-linoleic acid', 'Biotin', 'Calcium', 'Carbohydrate',
       'Chloride', 'Choline', 'Chromium', 'Copper', 'Fat', 'Fiber',
       'Fluoride', 'Folate', 'Iodine', 'Iron', 'Linoleic acid',
       'Magnesium', 'Manganese', 'Molybdenum', 'Monousaturated fat',
       'Niacin', 'Pantothenic acid', 'Phosphorus', 'Polyunsaturated fat',
       'Potassium', 'Protein', 'Riboflavin', 'Saturated fat', 'Selenium',
       'Sodium', 'Sugar', 'Thiamin', 'Vitamin A', 'Vitamin B6',
       'Vitamin B12', 'Vitamin C', 'Vitamin D', 'Vitamin E', 'Vitamin K',
       'Water', 'Zinc'], dtype=object)

# look up the wikipedia tables

In [10]:
def nutritional_facts(food) :
    
    html = urlopen("https://en.wikipedia.org/wiki/" + food) 
    res = BeautifulSoup(html.read(),"html5lib");

    nutritional_facts = {}

    table = res.find("table", {"class": "infobox nowrap"}) 
    table_body = table.find('tbody')


    rows = table_body.find_all('tr')
    for row in rows:

        nut_element = ""

        valid_name = False
        found_name = ""

        #find elements names
        elem = row.find('th') 

        if elem is not None :
            name = elem.find('a') 

            if name is None and elem.text is not None :
                name = elem


            if name is not None :

                #check if the name contains something interesting
                matching = [c.lower() in name.text.lower() for c in elements_names]
                valid_name  = sum(matching) >= 1            

                if valid_name :
                    matching_name = elements_names[matching.index(True)]
                    found_name = name.text
                    nut_element = matching_name


        #find element amount

        if valid_name :
            amount = row.find('td')

            if amount is not None :  

                #hard special case
                if found_name == "Vitamin A equiv." :
                    whole_string = amount.text
                    div_strings = [amount.find_all('div')[i].text for i in range(len(amount.find_all('div')))]
                    div_strings = sorted(div_strings, key = len, reverse = True)

                    for div_string in div_strings :
                        whole_string = whole_string.replace(div_string, '')


                    nutritional_facts[matching_name] = parse_amount(whole_string)

                elif 'div' not in str(amount) :

                    nutritional_facts[matching_name] = parse_amount(amount.text)

                else :              

                    #amount = parse_amount(amount.find_all('div')[0].text)
                    amount = parse_amount(amount.text)
                    nutritional_facts[matching_name] = amount

    return nutritional_facts

In [14]:
nutritional_facts("Jerusalem_artichoke")

{'Carbohydrate': '17440.0',
 'Sugar': '9600.0',
 'Fiber': '1600.0',
 'Fat': '10.0',
 'Protein': '2000.0',
 'Thiamin': '0.2',
 'Riboflavin': '0.06',
 'Niacin': '1.3',
 'Pantothenic acid': '0.397',
 'Vitamin B6': '0.077',
 'Folate': '0.013',
 'Vitamin C': '4',
 'Calcium': '14',
 'Iron': '3.4',
 'Magnesium': '17',
 'Phosphorus': '78',
 'Potassium': '429'}